In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from train import *
from tensorflow.keras import layers
from tensorflow.keras.layers import TimeDistributed, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import kapre
from kapre.composed import get_melspectrogram_layer
import tensorflow as tf
import os
from matplotlib import pyplot as plt

In [14]:
src_root = "npy_formal_dataset/train/"
npy_paths = glob('{}/**'.format(src_root), recursive=True)
npy_paths = sorted([x.replace(os.sep, '/') for x in npy_paths if "train.npy" in x])
print(npy_paths)
train_data = []
train_label = []
for i in npy_paths:
    #print(np.load(i).shape)
    train_data.append(np.abs(np.load(i)))
    cur_cls = os.path.split(i)[0].split('/')[-1]
    cur_label = [cur_cls for i in range(np.load(i).shape[0])]
    train_label += cur_label
train_data = np.stack(train_data)
train_label = np.stack(train_label)
train_data = train_data.reshape(train_data.shape[0]*train_data.shape[1],train_data.shape[2],train_data.shape[3])
print(train_data.shape)
print(train_label.shape)

['npy_formal_dataset/train/faucet/train.npy', 'npy_formal_dataset/train/hairdryer/train.npy', 'npy_formal_dataset/train/hood/train.npy', 'npy_formal_dataset/train/washbasin/train.npy']
(5440, 23, 641)
(5440,)


In [3]:
#Prepare for Training
src_root1 = "npy_formal_dataset/train/"
npy_paths1 = glob('{}/**'.format(src_root1), recursive=True)
npy_paths = npy_paths1+npy_paths2
npy_paths = [x for x in npy_paths if "Train" in x]
print(npy_paths)
train_data = []
train_label = []
for i in npy_paths:
    #print(np.load(i).shape)
    train_data.append(np.abs(np.load(i)))
    cur_cls = os.path.split(i)[-1].split('_')[1]
    cur_label = [cur_cls for i in range(np.load(i).shape[0])]
    train_label += cur_label
train_data = np.stack(train_data)
train_label = np.stack(train_label)
train_data = train_data.reshape(train_data.shape[0]*train_data.shape[1],train_data.shape[2],train_data.shape[3])
print(train_data.shape)

['npy_augmented_real/hairdryer_washbasin_8k_40k\\Real_hairdryer_Train.npy', 'npy_augmented_real/hairdryer_washbasin_8k_40k\\Real_washbasin_Train.npy', 'npy_augmented_real/hood_faucet_8k_40k\\Real_faucet_Train.npy', 'npy_augmented_real/hood_faucet_8k_40k\\Real_hood_Train.npy']
(3840, 23, 641)


In [16]:
classes = sorted(os.listdir(src_root))
print(classes)
num_object = len(classes)
num_instance = train_data.shape[0]
dim_instance = train_data.shape[1:]

['faucet', 'hairdryer', 'hood', 'washbasin']


In [17]:
for i in range(train_data.shape[0]):
    train_data[i,:,:] /= train_data[i,:,:].max()

In [18]:
print(train_data.max())
print(train_data.min())

1.0
1.3206194e-06


In [19]:
le = LabelEncoder()
le.fit(classes)

LabelEncoder()

In [20]:
labels = le.transform(train_label)

In [21]:
print(labels)

[0 0 0 ... 3 3 3]


In [22]:
trainset, valset, label_train, label_val = train_test_split(train_data,
                                                    labels,
                                                    test_size=0.25,
                                                    random_state=0)

In [23]:
class DataGenerator_npy(tf.keras.utils.Sequence):
    def __init__(self, data, labels, dim, n_classes,
                 batch_size=8, shuffle=True):
        self.data = data
        self.labels = labels
        self.dim = dim
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.shuffle = True
        self.on_epoch_end()


    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))


    def __getitem__(self, index):
        # Where is the input of index?
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        data_tmp = [self.data[k] for k in indexes]
        labels = [self.labels[k] for k in indexes]

        # generate a batch of time data
        X = np.empty((self.batch_size, *self.dim, 1), dtype=np.float32)
        # The format of Y gonna be: Y[0,:] = 0,1/1,0
        Y = np.empty((self.batch_size, self.n_classes), dtype=np.float32)

        for i, (instance, label) in enumerate(zip(data_tmp, labels)):
            X[i,] = instance.reshape((*self.dim,1))
            Y[i,] = to_categorical(label, num_classes=self.n_classes)

        return X, Y


    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [24]:
tg = DataGenerator_npy(trainset, label_train, dim_instance,num_object)

In [25]:
vg = DataGenerator_npy(valset, label_val, dim_instance,num_object)

In [26]:
def Conv1D_ultramic(N_CLASSES=4, dim=(23,641)):
    input = tf.keras.Input(shape=(*dim,1))
    x = LayerNormalization(axis=2, name='batch_norm')(input)
    x = TimeDistributed(layers.Conv1D(8, kernel_size=(4), activation='tanh'), name='td_conv_1d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(1,2), name='max_pool_2d_1')(x)
    x = TimeDistributed(layers.Conv1D(16, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(1,2), name='max_pool_2d_2')(x)
    x = TimeDistributed(layers.Conv1D(32, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_3')(x)
    x = TimeDistributed(layers.Conv1D(64, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_4')(x)
    x = TimeDistributed(layers.Conv1D(128, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_4')(x)
    x = layers.GlobalMaxPooling2D(name='global_max_pooling_2d')(x)
    x = layers.Dropout(rate=0.1, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)
    model = Model(inputs=input, outputs=o, name='1d_convolution')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def Conv2D_ultramic(N_CLASSES=4, dim=(23,641)):
    input = tf.keras.Input(shape=(*dim,1))
    x = LayerNormalization(axis=2, name='batch_norm')(input)
    x = layers.Conv2D(8, kernel_size=(7,7), activation='tanh', padding='same', name='conv2d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(1,2), padding='same', name='max_pool_2d_1')(x)
    x = layers.Conv2D(16, kernel_size=(5,5), activation='relu', padding='same', name='conv2d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(1,2), padding='same', name='max_pool_2d_2')(x)
    x = layers.Conv2D(16, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_3')(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_4')(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_4')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)
    model = Model(inputs=input, outputs=o, name='2d_convolution')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def LSTM_ultramic(N_CLASSES=4, dim=(23,641)):
    input = tf.keras.Input(shape=(*dim,1))
    x = LayerNormalization(axis=2, name='batch_norm')(input)
    x = TimeDistributed(layers.Reshape((-1,)), name='reshape')(x)
    s = TimeDistributed(layers.Dense(64, activation='tanh'),
                        name='td_dense_tanh')(x)
    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True),
                             name='bidirectional_lstm')(s)
    x = layers.concatenate([s, x], axis=2, name='skip_connection')
    x = layers.Dense(64, activation='relu', name='dense_1_relu')(x)
    x = layers.MaxPooling1D(name='max_pool_1d')(x)
    x = layers.Dense(32, activation='relu', name='dense_2_relu')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(32, activation='relu',
                         activity_regularizer=l2(0.001),
                         name='dense_3_relu')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)
    model = Model(inputs=input, outputs=o, name='long_short_term_memory')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [27]:
model = LSTM_ultramic()
csv_path = os.path.join('logs', '{}_test.csv'.format("lstm_overlapping"))

In [28]:
cp = ModelCheckpoint('models/{}.h5'.format("lstm_overlapping"), monitor='val_loss',
                         save_best_only=True, save_weights_only=False,
                         mode='auto', save_freq='epoch', verbose=1)
csv_logger = CSVLogger(csv_path, append=False)
model.fit(tg, validation_data=vg,
              epochs=30, verbose=1,
              callbacks=[csv_logger, cp])

Epoch 1/30
507/510 [============================>.] - ETA: 0s - loss: 0.1400 - accuracy: 0.9630
Epoch 00001: val_loss improved from inf to 0.03146, saving model to models\lstm_overlapping.h5
510/510 [==============================] - 3s 6ms/step - loss: 0.1394 - accuracy: 0.9632 - val_loss: 0.0315 - val_accuracy: 1.0000
Epoch 2/30
505/510 [============================>.] - ETA: 0s - loss: 0.0293 - accuracy: 1.0000
Epoch 00002: val_loss improved from 0.03146 to 0.02344, saving model to models\lstm_overlapping.h5
510/510 [==============================] - 3s 5ms/step - loss: 0.0293 - accuracy: 1.0000 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 3/30
504/510 [============================>.] - ETA: 0s - loss: 0.0225 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.02344 to 0.01916, saving model to models\lstm_overlapping.h5
510/510 [==============================] - 3s 5ms/step - loss: 0.0225 - accuracy: 1.0000 - val_loss: 0.0192 - val_accuracy: 1.0000
Epoch 4/30
509/510 [======